In [29]:
import heapq
import os
from functools import total_ordering

"""
Code for Huffman Coding, compression and decompression. 
Explanation at http://bhrigu.me/blog/2017/01/17/huffman-coding-python-implementation/
"""

@total_ordering
class HeapNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    # defining comparators less_than and equals
    def __lt__(self, other):
        return self.freq < other.freq

    def __eq__(self, other):
        if(other == None):
            return False
        if(not isinstance(other, HeapNode)):
            return False
        return self.freq == other.freq


class HuffmanCoding:
    def __init__(self, path):
        self.path = path
        self.heap = []
        self.codes = {}
        self.reverse_mapping = {}

    # functions for compression:

    def make_frequency_dict(self, text):
        frequency = {}
        for character in text:
            if not character in frequency:
                frequency[character] = 0
            frequency[character] += 1
        return frequency

    def make_heap(self, frequency):
        for key in frequency:
            node = HeapNode(key, frequency[key])
            heapq.heappush(self.heap, node)

    def merge_nodes(self):
        while(len(self.heap)>1):
            node1 = heapq.heappop(self.heap)
            node2 = heapq.heappop(self.heap)

            merged = HeapNode(None, node1.freq + node2.freq)
            merged.left = node1
            merged.right = node2

            heapq.heappush(self.heap, merged)


    def make_codes_helper(self, root, current_code):
        if(root == None):
            return

        if(root.char != None):
            self.codes[root.char] = current_code
            self.reverse_mapping[current_code] = root.char
            return

        self.make_codes_helper(root.left, current_code + "0")
        self.make_codes_helper(root.right, current_code + "1")


    def make_codes(self):
        root = heapq.heappop(self.heap)
        current_code = ""
        self.make_codes_helper(root, current_code)


    def get_encoded_text(self, text):
        encoded_text = ""
        for character in text:
            encoded_text += self.codes[character]
        return encoded_text


    def pad_encoded_text(self, encoded_text):
        extra_padding = 8 - len(encoded_text) % 8
        for i in range(extra_padding):
            encoded_text += "0"

        padded_info = "{0:08b}".format(extra_padding)
        encoded_text = padded_info + encoded_text
        return encoded_text


    def get_byte_array(self, padded_encoded_text):
        if(len(padded_encoded_text) % 8 != 0):
            print("Encoded text not padded properly")
            exit(0)

        b = bytearray()
        for i in range(0, len(padded_encoded_text), 8):
            byte = padded_encoded_text[i:i+8]
            b.append(int(byte, 2))
        return b


    def compress(self):
        filename, file_extension = os.path.splitext(self.path)
        output_path = filename + ".bin"

        with open(self.path, 'r+') as file, open(output_path, 'wb') as output:
            text = file.read()
            text = text.rstrip()

            frequency = self.make_frequency_dict(text)
            self.make_heap(frequency)
            self.merge_nodes()
            self.make_codes()

            encoded_text = self.get_encoded_text(text)
            padded_encoded_text = self.pad_encoded_text(encoded_text)

            b = self.get_byte_array(padded_encoded_text)
            output.write(bytes(b))
        
        print(self.reverse_mapping)
        print(self.codes)
        for k,v in self.reverse_mapping.items():
            print(type(k), 'corresponds to', type(v))


        print("Compressed")
        return output_path


    """ functions for decompression: """


    def remove_padding(self, padded_encoded_text):
        padded_info = padded_encoded_text[:8]
        extra_padding = int(padded_info, 2)

        padded_encoded_text = padded_encoded_text[8:] 
        encoded_text = padded_encoded_text[:-1*extra_padding]

        return encoded_text

    def decode_text(self, encoded_text):
        current_code = ""
        decoded_text = ""

        for bit in encoded_text:
            current_code += bit
            if(current_code in self.reverse_mapping):
                character = self.reverse_mapping[current_code]
                decoded_text += character
                current_code = ""

        return decoded_text


    def decompress(self, input_path):
        filename, file_extension = os.path.splitext(self.path)
        output_path = filename + "_decompressed" + ".txt"

        with open(input_path, 'rb') as file, open(output_path, 'w') as output:
            bit_string = ""

            byte = file.read(1)
            while(len(byte) > 0):
                byte = ord(byte)
                bits = bin(byte)[2:].rjust(8, '0')
                bit_string += bits
                byte = file.read(1)

            encoded_text = self.remove_padding(bit_string)

            decompressed_text = self.decode_text(encoded_text)
            
            output.write(decompressed_text)

        print("Decompressed")
        return output_path



In [30]:
# from huffman import HuffmanCoding

#input file path
path = '/Users/zhouhang/what.txt'

h = HuffmanCoding(path)

output_path = h.compress()
h.decompress(output_path)

{'0000': 'd', '00010': 'c', '0001100': 'x', '00011010': 'k', '000110110': '#', '000110111000': '<', '000110111001': 'E', '000110111010': 'j', '000110111011': 'D', '00011011110': '-', '00011011111': '%', '000111': '.', '001': 'e', '010000': '=', '0100010': 'q', '0100011': ',', '01001': 'p', '01010': 'f', '01011': '_', '011000': 'u', '011001000000': '*', '011001000001': '>', '01100100001': 'F', '0110010001': '}', '011001001': 'v', '0110010100': '{', '01100101010': 'C', '01100101011': '!', '011001011': '0', '0110011': 'g', '01101': 'a', '01110': 's', '011110': ')', '011111': '(', '10': ' ', '11000': 'o', '1100100': '"', '110010100': '8', '110010101': 'N', '110010110': '1', '1100101110': 'H', '1100101111': "'", '110011': 'l', '11010': '\n', '11011': 'n', '1110000': 'y', '1110001': ':', '1110010000': 'w', '1110010001': '2', '111001001': '+', '111001010': ']', '111001011': '[', '1110011': 'm', '11101': 't', '11110': 'r', '1111100': 'b', '1111101': 'h', '111111': 'i'}
{'d': '0000', 'c': '0001

'/Users/zhouhang/what_decompressed.txt'

In [37]:
d = {'x': 1, 'y': 2, 'z': 3} 
b = bytes(str(d), 'utf-8')
b


b"{'x': 1, 'y': 2, 'z': 3}"

In [26]:
for k,v in d.items():
    print(type(k), 'corresponds to', type(v))

<class 'str'> corresponds to <class 'int'>
<class 'str'> corresponds to <class 'int'>
<class 'str'> corresponds to <class 'int'>
